In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa

import sys
sys.path.append('/Users/zachkeskinen/Documents/spicy-snow/')

import pickle
with open('/Users/zachkeskinen/Documents/spicy-snow/data/banner_lidar.pkl', 'rb') as f:
    ds = pickle.load(f)
# ds = ds.isel(time = slice(0, 3))

In [ ]:
import sys
sys.path.append('/Users/zachkeskinen/Documents/spicy-snow/')
from spicy_snow.processing.snow_index import calc_snow_index

import numpy as np
import xarray as xr

backscatter = np.random.randn(10, 10, 6, 3)
deltaGamma = np.random.randn(10, 10 , 6)
times = [np.datetime64(t) for t in ['2020-01-01','2020-01-02', '2020-01-07','2020-01-08', '2020-01-14', '2020-01-15']]
ims = np.full((10, 10, 6), 4)
x = np.linspace(0, 9, 10)
y = np.linspace(10, 19, 10)
lon, lat = np.meshgrid(x, y)

test_ds = xr.Dataset(
    data_vars = dict(
        s1 = (["x", "y", "time", "band"], backscatter),
        deltaGamma = (["x", "y", "time"], deltaGamma),
        ims = (["x", "y", "time"], ims)
    ),

    coords = dict(
        lon = (["x", "y"], lon),
        lat = (["x", "y"], lat),
        band = ['VV', 'VH', 'inc'],
        time = times,
        relative_orbit = (["time"], [24, 1, 24, 1, 24, 1])))

test_ds['deltaGamma'].loc[dict(time = '2020-01-01')] = np.nan
test_ds['deltaGamma'].loc[dict(time = '2020-01-02')] = np.nan

ds = calc_snow_index(test_ds)

In [ ]:
ds['snow_index'].isel(time = 2)[0,:]

In [ ]:
ds['deltaGamma'].isel(time = 3)[0,:]

In [ ]:
ds['snow_index'].isel(time = 3)[0,:]